In [27]:
import numpy 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
import numpy as np

In [28]:
from data_prep import DataPrep

In [29]:
data_location = "./data/compas-scores-two-years.csv"
compas_score_full = pd.read_csv(data_location)
data_prepper = DataPrep(compas_score_full)
X, y, X_cols = data_prepper.get()

features = [c for c in X]

categorical_feature_name = ['two_year_recid', 'c_charge_degree_F', 'c_charge_degree_M',\
                            'sex_Female', 'sex_Male', 'race']

categorical_feature_indcs = [features.index(c) for c in categorical_feature_name]
race_indc = features.index('race')

X=X.to_numpy()
#make all columns float
X = X.astype(float)
y=np.array(y)



C:\Users\Tabea Heusel\PycharmProjects\Lime-Enhancements\data_prep.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  compas_df['length_of_stay'] = (pd.to_datetime(compas_df['c_jail_out']) - pd.to_datetime(compas_df['c_jail_in'])).dt.days


In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
naive_bayes_model = GaussianNB()
naive_bayes_model.fit(X_train, y_train)

y_pred = naive_bayes_model.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.8307692307692308
Classification Report:
              precision    recall  f1-score   support

           0       0.54      0.42      0.47       224
           1       0.88      0.92      0.90      1011

    accuracy                           0.83      1235
   macro avg       0.71      0.67      0.69      1235
weighted avg       0.82      0.83      0.82      1235



In [32]:
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

Confusion Matrix:
[[ 94 130]
 [ 79 932]]


### Standard deviation

In [33]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_std = scaler.fit_transform(std_per_variable.values.reshape(-1, 1))

scaled_std_df = pd.DataFrame({'Feature': std_per_variable.index, 'Scaled Standard Deviation': scaled_std.flatten()})
scaled_std_df = scaled_std_df.sort_values(by='Scaled Standard Deviation', ascending=False)

print("Sorted Scaled Standard Deviation:")
print(scaled_std_df)

In [34]:
from sklearn.feature_selection import mutual_info_classif

### Mutual information

In [35]:
information_gain = mutual_info_classif(X, y, discrete_features='auto', random_state=42)

scaled_information_gain = information_gain / information_gain.max()

information_gain_df = pd.DataFrame({'Feature': X_cols, 'Scaled Information Gain': scaled_information_gain})
information_gain_df = information_gain_df.sort_values(by='Scaled Information Gain', ascending=False)

print("\nScaled Information Gain:")
print(information_gain_df)


Scaled Information Gain:
             Feature  Scaled Information Gain
2       priors_count                 1.000000
1     two_year_recid                 0.741626
8               race                 0.431101
0                age                 0.430770
3     length_of_stay                 0.417032
4  c_charge_degree_F                 0.217664
5  c_charge_degree_M                 0.173355
6         sex_Female                 0.086700
7           sex_Male                 0.078037


### Changing in LIME base 

if model_regressor is None:
            model_regressor = GaussianNB(random_state=self.random_state)
easy_model = model_regressor
easy_model.fit(neighborhood_data[:, used_features],
                       labels_column, sample_weight=weights)
prediction_score = easy_model.score(
            neighborhood_data[:, used_features],
            labels_column, sample_weight=weights)

local_pred = easy_model.predict(neighborhood_data[0, used_features].reshape(1, -1))
information_gain = mutual_info_classif(X, y, discrete_features='auto', random_state=42)

scaled_information_gain = information_gain / information_gain.max()

information_gain_df = pd.DataFrame({'Feature': X.columns, 'Scaled Information Gain': scaled_information_gain})
information_gain_df = information_gain_df.sort_values(by='Scaled Information Gain', ascending=False)

easy_model_coef=information_gain_df
if self.verbose:
    print('Prediction_local', local_pred, )
    print('Right:', neighborhood_labels[0, label])
return (sorted(zip(used_features, easy_model_coef),
                       key=lambda x: np.abs(x[1]), reverse=True),
                prediction_score, local_pred)

In [36]:
from model_builder import ModelBuild


In [37]:
print(X_cols)
modelBuilder = ModelBuild('DecisionTree')
model, accuracy, cm = modelBuilder.train_eval(X_train, y_train, X_test, y_test)
explainer_lime = modelBuilder.naive_bayes_explain(X_train, X_cols)
print(X_cols)

instance_no = 76
#instance to explain (for easy column viewing converted to pandas)
print(f"Instance to be explained:\n{pd.DataFrame(X_test, columns=X_cols).iloc[instance_no]}")


['age', 'two_year_recid', 'priors_count', 'length_of_stay', 'c_charge_degree_F', 'c_charge_degree_M', 'sex_Female', 'sex_Male', 'race']
['age', 'two_year_recid', 'priors_count', 'length_of_stay', 'c_charge_degree_F', 'c_charge_degree_M', 'sex_Female', 'sex_Male', 'race']
Instance to be explained:
age                  22.0
two_year_recid        0.0
priors_count          1.0
length_of_stay        4.0
c_charge_degree_F     1.0
c_charge_degree_M     0.0
sex_Female            0.0
sex_Male              1.0
race                  1.0
Name: 76, dtype: float64


In [38]:
exp_inst = explainer_lime.explain_instance(X_test[instance_no], model.predict_proba, num_features=5)

(5000, 5)
(5000,)


ValueError: Unknown label type: (array([0.        , 0.25      , 0.33333333, 0.44444444, 0.5       ,
       0.55555556, 0.625     , 0.66666667, 0.71428571, 0.75      ,
       0.8       , 0.83333333, 0.875     , 1.        ]),)